In [24]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle

from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.metrics import dtw_subsequence_path

# import xgboost as xgb

In [25]:
dataset_folder = "../all"
!ls {dataset_folder}

data_note.pdf	       test_set_metadata.csv  training_set_metadata.csv
sample_submission.csv  test_set_sample.csv
test_set.csv	       training_set.csv


In [26]:
def read_pickle(name):
    with open(name, 'rb') as f:
        data = pickle.load(f)
    return data

In [27]:
xg_model = read_pickle("xg_model")
shapelets = read_pickle("shapelets")
print(shapelets.shape)

(36,)


/home/frankl1/.env/plasticc/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.3 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [28]:
classes = np.array([ 6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95, 99])
classes

array([ 6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95, 99])

In [29]:
def normalize(ts, ts_err):
    ts /= (ts_err + 1) # +1 to avoid zero division
    ts = np.nan_to_num(ts)
    ts = TimeSeriesScalerMinMax().fit_transform(ts)
    return ts

In [30]:
def distanceToShapelet(ts, shapelet):
    path, dist = dtw_subsequence_path(shapelet, ts)
    return dist

In [31]:
def distanceToShapelets(ts, shapelets=shapelets):
    return [distanceToShapelet(ts, shapelet) for shapelet in shapelets]

In [32]:
step = 1
nb_of_passband = 6
max_mjd = 1094
print(nb_of_passband, max_mjd, sep="\n")

6
1094


In [33]:
def format_time_series(data):
    X = []
    X_err = []
    s0 = int(data.shape[0] / nb_of_passband)
    x, x_err = [], []
    for p in range(nb_of_passband):
        x = np.append(x, data[data.passband==p].flux.values)
        x_err = np.append(x_err, data[data.passband==p].flux_err.values)
    X.append(x.reshape(s0, nb_of_passband))
    X_err.append(x.reshape(s0, nb_of_passband))
    return np.array(X), np.array(X_err)

In [34]:
def mjdToPredict(mjdToExclude, minMjd, maxMjd, step):
    return np.array(list(set(range(minMjd, maxMjd+1, step)) - set(mjdToExclude)))

In [35]:
def fill_missing_flux(objectDf, maxMjd, step):
    objectDf.mjd = objectDf[["mjd"]].astype(np.int64)
    objectDf.mjd = objectDf.mjd - objectDf.mjd.min()
    instance = objectDf.groupby(["passband", "mjd"]).mean().reset_index()
    meanByPassband = instance.groupby("passband").mean()[["flux", "flux_err"]]
    frames = [instance]
    for p in range(6):
        toExclude = instance[instance.passband==p].mjd
        toPredict = mjdToPredict(toExclude, 0, maxMjd, step)
        nb = len(toPredict)
        frames.append(pd.DataFrame({
            'mjd': toPredict,
            'flux': [meanByPassband.loc[p].flux]*nb,
            'flux_err': [meanByPassband.loc[p].flux_err]*nb,
            'detected': [-1] * nb,
            'passband': [p]*nb,
            'object_id': [objectDf.object_id.values[0]]*nb
        }))
    return pd.concat(frames, sort=False)

In [36]:
columns = ["object_id"]
columns.extend([f"class_{c}" for c in classes])
submission = pd.DataFrame(columns = columns)
submission

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99


In [37]:
def predict_batch(test_df):
    instance = fill_missing_flux(test_df, max_mjd, step)
    ts, ts_err = format_time_series(instance)
    ts_norm = normalize(ts, ts_err)
    dist_vec = distanceToShapelets(ts_norm[0], shapelets)
    preds = xg_model.predict_proba([dist_vec])[0]
    preds = np.concatenate(([test_df.object_id.values[0]], preds, [0]))
    submission.loc[submission.shape[0]] = preds

In [38]:
max_nb_of_chunk = 10
start_at = 0
print("Starting at:", start_at)

Starting at: 0


In [43]:
def predict_test_set(start_at=0):
    test_example = []
    curr_id = None
    i = 0
    for chunk in pd.read_csv(f"{dataset_folder}/test_set.csv", chunksize=1000, skiprows=list(range(1, start_at+1))):
        i += 1
        if curr_id is None:
            curr_id = chunk.object_id.values[0]
        print(f"Batch {i}")
        test_example.append(chunk.copy())
        if chunk.object_id.values[-1] != curr_id:
            tail = pd.concat(test_example)
            obj_ids = tail.object_id.unique()
            while obj_ids.shape[0] > 1:
                one_object_df = tail[tail.object_id==obj_ids[0]]
                predict_batch(one_object_df)
                obj_ids = np.delete(obj_ids, 0)
                start_at += one_object_df.shape[0]
            test_example = [tail[tail.object_id.isin(obj_ids)]]
            current_id = None if tail.shape[0] == 0 else tail.object_id.values[0]    
#         if i != max_nb_of_chunk:
#             print("EOF")
#             break
    test_df = pd.concat(test_example)
    predict_batch(test_df)
    print("End at:", start_at)

In [44]:
%%time
predict_test_set(start_at=start_at)

Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Batch 10
Batch 11
Batch 12
Batch 13
Batch 14
Batch 15
Batch 16
Batch 17
Batch 18
Batch 19
Batch 20
Batch 21
Batch 22
Batch 23
Batch 24
Batch 25
Batch 26
Batch 27
Batch 28
Batch 29
Batch 30
Batch 31
Batch 32
Batch 33
Batch 34
Batch 35
Batch 36
Batch 37
Batch 38
Batch 39
Batch 40
Batch 41
Batch 42
Batch 43
Batch 44
Batch 45
Batch 46
Batch 47
Batch 48
Batch 49
Batch 50
Batch 51
Batch 52
Batch 53
Batch 54
Batch 55
Batch 56
Batch 57
Batch 58
Batch 59
Batch 60
Batch 61
Batch 62
Batch 63
Batch 64
Batch 65
Batch 66
Batch 67
Batch 68
Batch 69
Batch 70
Batch 71
Batch 72
Batch 73
Batch 74
Batch 75
Batch 76
Batch 77
Batch 78
Batch 79
Batch 80
Batch 81
Batch 82
Batch 83
Batch 84
Batch 85
Batch 86
Batch 87
Batch 88
Batch 89
Batch 90
Batch 91
Batch 92
Batch 93
Batch 94
Batch 95
Batch 96
Batch 97
Batch 98
Batch 99
Batch 100
Batch 101
Batch 102
Batch 103
Batch 104
Batch 105
Batch 106
Batch 107
Batch 108
Batch 109
Batch 110
Batch 11

Batch 832
Batch 833
Batch 834
Batch 835
Batch 836
Batch 837
Batch 838
Batch 839
Batch 840
Batch 841
Batch 842
Batch 843
Batch 844
Batch 845
Batch 846
Batch 847
Batch 848
Batch 849
Batch 850
Batch 851
Batch 852
Batch 853
Batch 854
Batch 855
Batch 856
Batch 857
Batch 858
Batch 859
Batch 860
Batch 861
Batch 862
Batch 863
Batch 864
Batch 865
Batch 866
Batch 867
Batch 868
Batch 869
Batch 870
Batch 871
Batch 872
Batch 873
Batch 874
Batch 875
Batch 876
Batch 877
Batch 878
Batch 879
Batch 880
Batch 881
Batch 882
Batch 883
Batch 884
Batch 885
Batch 886
Batch 887
Batch 888
Batch 889
Batch 890
Batch 891
Batch 892
Batch 893
Batch 894
Batch 895
Batch 896
Batch 897
Batch 898
Batch 899
Batch 900
Batch 901
Batch 902
Batch 903
Batch 904
Batch 905
Batch 906
Batch 907
Batch 908
Batch 909
Batch 910
Batch 911
Batch 912
Batch 913
Batch 914
Batch 915
Batch 916
Batch 917
Batch 918
Batch 919
Batch 920
Batch 921
Batch 922
Batch 923
Batch 924
Batch 925
Batch 926
Batch 927
Batch 928
Batch 929
Batch 930
Batch 931


Batch 1593
Batch 1594
Batch 1595
Batch 1596
Batch 1597
Batch 1598
Batch 1599
Batch 1600
Batch 1601
Batch 1602
Batch 1603
Batch 1604
Batch 1605
Batch 1606
Batch 1607
Batch 1608
Batch 1609
Batch 1610
Batch 1611
Batch 1612
Batch 1613
Batch 1614
Batch 1615
Batch 1616
Batch 1617
Batch 1618
Batch 1619
Batch 1620
Batch 1621
Batch 1622
Batch 1623
Batch 1624
Batch 1625
Batch 1626
Batch 1627
Batch 1628
Batch 1629
Batch 1630
Batch 1631
Batch 1632
Batch 1633
Batch 1634
Batch 1635
Batch 1636
Batch 1637
Batch 1638
Batch 1639
Batch 1640
Batch 1641
Batch 1642
Batch 1643
Batch 1644
Batch 1645
Batch 1646
Batch 1647
Batch 1648
Batch 1649
Batch 1650
Batch 1651
Batch 1652
Batch 1653
Batch 1654
Batch 1655
Batch 1656
Batch 1657
Batch 1658
Batch 1659
Batch 1660
Batch 1661
Batch 1662
Batch 1663
Batch 1664
Batch 1665
Batch 1666
Batch 1667
Batch 1668
Batch 1669
Batch 1670
Batch 1671
Batch 1672
Batch 1673
Batch 1674
Batch 1675
Batch 1676
Batch 1677
Batch 1678
Batch 1679
Batch 1680
Batch 1681
Batch 1682
Batch 1683

Batch 2338
Batch 2339
Batch 2340
Batch 2341
Batch 2342
Batch 2343
Batch 2344
Batch 2345
Batch 2346
Batch 2347
Batch 2348
Batch 2349
Batch 2350
Batch 2351
Batch 2352
Batch 2353
Batch 2354
Batch 2355
Batch 2356
Batch 2357
Batch 2358
Batch 2359
Batch 2360
Batch 2361
Batch 2362
Batch 2363
Batch 2364
Batch 2365
Batch 2366
Batch 2367
Batch 2368
Batch 2369
Batch 2370
Batch 2371
Batch 2372
Batch 2373
Batch 2374
Batch 2375
Batch 2376
Batch 2377
Batch 2378
Batch 2379
Batch 2380
Batch 2381
Batch 2382
Batch 2383
Batch 2384
Batch 2385
Batch 2386
Batch 2387
Batch 2388
Batch 2389
Batch 2390
Batch 2391
Batch 2392
Batch 2393
Batch 2394
Batch 2395
Batch 2396
Batch 2397
Batch 2398
Batch 2399
Batch 2400
Batch 2401
Batch 2402
Batch 2403
Batch 2404
Batch 2405
Batch 2406
Batch 2407
Batch 2408
Batch 2409
Batch 2410
Batch 2411
Batch 2412
Batch 2413
Batch 2414
Batch 2415
Batch 2416
Batch 2417
Batch 2418
Batch 2419
Batch 2420
Batch 2421
Batch 2422
Batch 2423
Batch 2424
Batch 2425
Batch 2426
Batch 2427
Batch 2428

Batch 3083
Batch 3084
Batch 3085
Batch 3086
Batch 3087
Batch 3088
Batch 3089
Batch 3090
Batch 3091
Batch 3092
Batch 3093
Batch 3094
Batch 3095
Batch 3096
Batch 3097
Batch 3098
Batch 3099
Batch 3100
Batch 3101
Batch 3102
Batch 3103
Batch 3104
Batch 3105
Batch 3106
Batch 3107
Batch 3108
Batch 3109
Batch 3110
Batch 3111
Batch 3112
Batch 3113
Batch 3114
Batch 3115
Batch 3116
Batch 3117
Batch 3118
Batch 3119
Batch 3120
Batch 3121
Batch 3122
Batch 3123
Batch 3124
Batch 3125
Batch 3126
Batch 3127
Batch 3128
Batch 3129
Batch 3130
Batch 3131
Batch 3132
Batch 3133
Batch 3134
Batch 3135
Batch 3136
Batch 3137
Batch 3138
Batch 3139
Batch 3140
Batch 3141
Batch 3142
Batch 3143
Batch 3144
Batch 3145
Batch 3146
Batch 3147
Batch 3148
Batch 3149
Batch 3150
Batch 3151
Batch 3152
Batch 3153
Batch 3154
Batch 3155
Batch 3156
Batch 3157
Batch 3158
Batch 3159
Batch 3160
Batch 3161
Batch 3162
Batch 3163
Batch 3164
Batch 3165
Batch 3166
Batch 3167
Batch 3168
Batch 3169
Batch 3170
Batch 3171
Batch 3172
Batch 3173

...

Batch 55098
Batch 55099
Batch 55100
Batch 55101
Batch 55102
Batch 55103
Batch 55104
Batch 55105
Batch 55106
Batch 55107
Batch 55108
Batch 55109
Batch 55110
Batch 55111
Batch 55112
Batch 55113
Batch 55114
Batch 55115
Batch 55116
Batch 55117
Batch 55118
Batch 55119
Batch 55120
Batch 55121
Batch 55122
Batch 55123
Batch 55124
Batch 55125
Batch 55126
Batch 55127
Batch 55128
Batch 55129
Batch 55130
Batch 55131
Batch 55132
Batch 55133
Batch 55134
Batch 55135
Batch 55136
Batch 55137
Batch 55138
Batch 55139
Batch 55140
Batch 55141
Batch 55142
Batch 55143
Batch 55144
Batch 55145
Batch 55146
Batch 55147
Batch 55148
Batch 55149
Batch 55150
Batch 55151
Batch 55152
Batch 55153
Batch 55154
Batch 55155
Batch 55156
Batch 55157
Batch 55158
Batch 55159
Batch 55160
Batch 55161
Batch 55162
Batch 55163
Batch 55164
Batch 55165
Batch 55166
Batch 55167
Batch 55168
Batch 55169
Batch 55170
Batch 55171
Batch 55172
Batch 55173
Batch 55174
Batch 55175
Batch 55176
Batch 55177
Batch 55178
Batch 55179
Batch 55180
Batc

Batch 55781
Batch 55782
Batch 55783
Batch 55784
Batch 55785
Batch 55786
Batch 55787
Batch 55788
Batch 55789
Batch 55790
Batch 55791
Batch 55792
Batch 55793
Batch 55794
Batch 55795
Batch 55796
Batch 55797
Batch 55798
Batch 55799
Batch 55800
Batch 55801
Batch 55802
Batch 55803
Batch 55804
Batch 55805
Batch 55806
Batch 55807
Batch 55808
Batch 55809
Batch 55810
Batch 55811
Batch 55812
Batch 55813
Batch 55814
Batch 55815
Batch 55816
Batch 55817
Batch 55818
Batch 55819
Batch 55820
Batch 55821
Batch 55822
Batch 55823
Batch 55824
Batch 55825
Batch 55826
Batch 55827
Batch 55828
Batch 55829
Batch 55830
Batch 55831
Batch 55832
Batch 55833
Batch 55834
Batch 55835
Batch 55836
Batch 55837
Batch 55838
Batch 55839
Batch 55840
Batch 55841
Batch 55842
Batch 55843
Batch 55844
Batch 55845
Batch 55846
Batch 55847
Batch 55848
Batch 55849
Batch 55850
Batch 55851
Batch 55852
Batch 55853
Batch 55854
Batch 55855
Batch 55856
Batch 55857
Batch 55858
Batch 55859
Batch 55860
Batch 55861
Batch 55862
Batch 55863
Batc

Batch 56464
Batch 56465
Batch 56466
Batch 56467
Batch 56468
Batch 56469
Batch 56470
Batch 56471
Batch 56472
Batch 56473
Batch 56474
Batch 56475
Batch 56476
Batch 56477
Batch 56478
Batch 56479
Batch 56480
Batch 56481
Batch 56482
Batch 56483
Batch 56484
Batch 56485
Batch 56486
Batch 56487
Batch 56488
Batch 56489
Batch 56490
Batch 56491
Batch 56492
Batch 56493
Batch 56494
Batch 56495
Batch 56496
Batch 56497
Batch 56498
Batch 56499
Batch 56500
Batch 56501
Batch 56502
Batch 56503
Batch 56504
Batch 56505
Batch 56506
Batch 56507
Batch 56508
Batch 56509
Batch 56510
Batch 56511
Batch 56512
Batch 56513
Batch 56514
Batch 56515
Batch 56516
Batch 56517
Batch 56518
Batch 56519
Batch 56520
Batch 56521
Batch 56522
Batch 56523
Batch 56524
Batch 56525
Batch 56526
Batch 56527
Batch 56528
Batch 56529
Batch 56530
Batch 56531
Batch 56532
Batch 56533
Batch 56534
Batch 56535
Batch 56536
Batch 56537
Batch 56538
Batch 56539
Batch 56540
Batch 56541
Batch 56542
Batch 56543
Batch 56544
Batch 56545
Batch 56546
Batc

Batch 57147
Batch 57148
Batch 57149
Batch 57150
Batch 57151
Batch 57152
Batch 57153
Batch 57154
Batch 57155
Batch 57156
Batch 57157
Batch 57158
Batch 57159
Batch 57160
Batch 57161
Batch 57162
Batch 57163
Batch 57164
Batch 57165
Batch 57166
Batch 57167
Batch 57168
Batch 57169
Batch 57170
Batch 57171
Batch 57172
Batch 57173
Batch 57174
Batch 57175
Batch 57176
Batch 57177
Batch 57178
Batch 57179
Batch 57180
Batch 57181
Batch 57182
Batch 57183
Batch 57184
Batch 57185
Batch 57186
Batch 57187
Batch 57188
Batch 57189
Batch 57190
Batch 57191
Batch 57192
Batch 57193
Batch 57194
Batch 57195
Batch 57196
Batch 57197
Batch 57198
Batch 57199
Batch 57200
Batch 57201
Batch 57202
Batch 57203
Batch 57204
Batch 57205
Batch 57206
Batch 57207
Batch 57208
Batch 57209
Batch 57210
Batch 57211
Batch 57212
Batch 57213
Batch 57214
Batch 57215
Batch 57216
Batch 57217
Batch 57218
Batch 57219
Batch 57220
Batch 57221
Batch 57222
Batch 57223
Batch 57224
Batch 57225
Batch 57226
Batch 57227
Batch 57228
Batch 57229
Batc

KeyboardInterrupt: 

In [50]:
submission.object_id = submission.object_id.astype(np.int32, inplace=True)
submission

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
0,13,0.003123,0.127717,0.037716,0.106163,0.022257,0.000123,0.118859,0.003935,0.117977,0.020423,0.049972,0.338819,0.045128,0.007789,0.0
1,14,0.010370,0.050463,0.098598,0.208956,0.025363,0.000085,0.032586,0.000792,0.096454,0.016375,0.040242,0.371861,0.028590,0.019264,0.0
2,17,0.019215,0.101485,0.049902,0.152822,0.012664,0.000019,0.050815,0.006143,0.072409,0.008670,0.065200,0.373362,0.020666,0.066628,0.0
3,23,0.014167,0.039756,0.074147,0.142434,0.004951,0.000381,0.182278,0.001510,0.056391,0.042536,0.012511,0.405591,0.015290,0.008057,0.0
4,34,0.005722,0.099124,0.049372,0.172879,0.008973,0.000013,0.046392,0.001199,0.071498,0.012778,0.019405,0.482519,0.006462,0.023664,0.0
5,35,0.015494,0.027120,0.107470,0.130369,0.005349,0.000405,0.081119,0.000812,0.093313,0.008302,0.308226,0.186731,0.020014,0.015274,0.0
6,43,0.030117,0.030260,0.018145,0.044187,0.006645,0.000464,0.088345,0.000175,0.054351,0.006872,0.093425,0.517742,0.035533,0.073739,0.0
7,50,0.001389,0.074965,0.328837,0.151929,0.003638,0.000092,0.002684,0.009831,0.208808,0.013527,0.019825,0.152929,0.031008,0.000537,0.0
8,60,0.003047,0.045587,0.150960,0.101506,0.024903,0.000444,0.039348,0.000892,0.199954,0.040280,0.023181,0.308047,0.048278,0.013572,0.0
9,69,0.002013,0.055724,0.049105,0.225032,0.003054,0.000038,0.094401,0.001132,0.110527,0.032075,0.022757,0.350589,0.005896,0.047657,0.0


In [47]:
submission.to_csv("submission.csv", index=False)

KeyboardInterrupt: 

In [ ]:
453_653_105